In [1]:
%load_ext autoreload
%autoreload 2

## Evaluation d'un modèle text2SQL 

In [2]:
from random import sample
import pandas as pd
import sqlite3
import sagemaker
from datasets import load_from_disk
from transformers import AutoTokenizer, AutoModelForCausalLM
from scripts.evaluate import evaluate_query
from scripts.llm import LLM, prepare_query, generate_sql

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


Parcourir toutes chaque ligne du dataset de test, générer le SQL. Evaluer par rapport à la ground truth.

Exemple d'usage

In [3]:
sess = sagemaker.Session()
sagemaker_session_bucket = sess.default_bucket()

In [4]:
lm_dataset_test = load_from_disk(f's3://{sagemaker_session_bucket}/text_2_sql/dataset/test')

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/fsspec/registry.py:279: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


In [5]:
dbfile = "sqllite/atis-db.sqlite"
con = sqlite3.connect(dbfile)

In [6]:
evaluate_query("SELECT * FROM aircraft LIMIT 1", "SELECT * FROM aircraft LIMIT 1", con)

{'valid_llm_query': True,
 'valid_llm_query_result': True,
 'faster_or_equal_llm_query_exec_time': False}

Vous retourne les résultats sur 3 métriques : {'valid_llm_query': True,
 'valid_llm_query_result': True,
 'faster_or_equal_llm_query_exec_time': True}

## Sortir les métriques avec et sans finetuning

In [7]:
model_id = "Qwen/Qwen2-1.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    "./finetuned_llm/qwen2",
    torch_dtype="auto",
    device_map="auto",
    max_memory={0: "5GB"}
)
llm = LLM(model=model, tokenizer=tokenizer)

In [8]:
random_test_idx = sample(range(len(lm_dataset_test)), 5)
model_evaluation = []
for idx in random_test_idx:
    query = prepare_query(lm_dataset_test[idx]["instruction"], "./prompt.json")
    sql_generated = generate_sql(query, llm)
    model_evaluation.append(evaluate_query(sql_generated, lm_dataset_test[idx]["output"], con))

In [9]:
df = pd.DataFrame(model_evaluation)

In [10]:
df

,valid_llm_query,valid_llm_query_result,faster_or_equal_llm_query_exec_time
0,True,False,False
1,True,True,True
2,True,False,False
3,True,False,False
4,True,False,False


In [11]:
print("Pourcentage des syntaxes des requêtes générées correctes: {:.2f} %".format((df["valid_llm_query"].sum()/df.shape[0])*100))
print("Pourcentage des résultats générés corrects: {:.2f} %".format((df["valid_llm_query_result"].sum()/df.shape[0])*100))
print("Pourcentage du temps d'exécution des requêtes générées moins long: {:.2f} %".format((df["faster_or_equal_llm_query_exec_time"].sum()/df.shape[0])*100))

Pourcentage des syntaxes des requêtes générées correctes: 100.00 %
Pourcentage des résultats générés corrects: 20.00 %
Pourcentage du temps d'exécution des requêtes générées moins long: 20.00 %
